In [1]:
import pandas as pd
import numpy as np
import os
import glob
import  PIL
from PIL import Image
import cv2
import time
import matplotlib.pyplot as plt
import scipy
from scipy import ndimage
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from data_cleaning import *
from keras import models
from keras import layers

Using TensorFlow backend.


## Image Preprocessing

In [2]:
# this function exists in data-cleaning.py
test_algae = "split/test/algae/*"
test_not_algae = "split/test/not_algae/*"
get_img_stats(test_algae)
get_img_stats(test_not_algae)

train_algae = "split/train/algae/*"
train_not_algae = "split/train/not_algae/*"
get_img_stats(train_algae)
get_img_stats(train_not_algae)

val_algae = "split/validation/algae/*"
val_not_algae = "split/validation/not_algae/*"
get_img_stats(val_algae)
get_img_stats(val_not_algae)
print("\n")
print("The minimumn image size is: 140 x 100! We will rescale to this size")

images appended!
(140, 100)
images appended!
(160, 237)
images appended!
(140, 100)
images appended!
(153, 110)
images appended!
(140, 100)
images appended!
(220, 165)


The minimumn image size is: 140 x 100! We will rescale to this size


In [3]:
# reshape the images using Keras 

# get all the data in the directory split/test (230 images), and reshape them
test_generator = ImageDataGenerator(rescale=1./255).flow_from_directory('split/test/', 
        target_size=(140, 100), batch_size = 230)

# get all the data in the directory split/train (570 images), and reshape them
train_generator = ImageDataGenerator(rescale=1./255).flow_from_directory(
    'split/train/', 
    target_size=(140, 100), batch_size=570)

# get all the data in the directory split/validation (160 images), and reshape them
val_generator = ImageDataGenerator(rescale=1./255).flow_from_directory(
    'split/validation/', 
    target_size=(140, 100), batch_size = 160)

Found 228 images belonging to 2 classes.
Found 570 images belonging to 2 classes.
Found 160 images belonging to 2 classes.


In [4]:
# now we'll move on to creating the data sets w/ labels 
train_images, train_labels = next(train_generator)
test_images, test_labels = next(test_generator)
val_images, val_labels = next(val_generator)

In [5]:
# lets check our dataset numbers again 
m_train = train_images.shape[0]
num_px = train_images.shape[1]
m_test = test_images.shape[0]
m_val = val_images.shape[0]

print ("Number of training samples: " + str(m_train))
print ("Number of testing samples: " + str(m_test))
print ("Number of validation samples: " + str(m_val))
print ("train_images shape: " + str(train_images.shape))
print ("train_labels shape: " + str(train_labels.shape))
print ("test_images shape: " + str(test_images.shape))
print ("test_labels shape: " + str(test_labels.shape))
print ("val_images shape: " + str(val_images.shape))
print ("val_labels shape: " + str(val_labels.shape))

shuffle = True

Number of training samples: 570
Number of testing samples: 228
Number of validation samples: 160
train_images shape: (570, 140, 100, 3)
train_labels shape: (570, 2)
test_images shape: (228, 140, 100, 3)
test_labels shape: (228, 2)
val_images shape: (160, 140, 100, 3)
val_labels shape: (160, 2)


In [6]:
train_img = train_images.reshape(train_images.shape[0], -1)
test_img = test_images.reshape(test_images.shape[0], -1)
val_img = val_images.reshape(val_images.shape[0], -1)

print(train_img.shape)
print(test_img.shape)
print(val_img.shape)

(570, 42000)
(228, 42000)
(160, 42000)


In [8]:
train_y = np.reshape(train_labels[:,0], (570,1))
test_y = np.reshape(test_labels[:,0], (228,1))
val_y = np.reshape(val_labels[:,0], (160,1))

print(train_y.shape)
print(test_y.shape)
print(val_y.shape)

(570, 1)
(228, 1)
(160, 1)


In [ ]:
from sklearn.model_selection import train_test_split